In [4]:
import sys
sys.path.append("../src")

In [5]:
import pandas as pd
from labels import LABELS, label2id

In [6]:
df = pd.read_csv("../data/processed/all_clauses_clean.csv")

In [7]:
df = df[df["label"].isin(LABELS)].copy()

In [8]:
#Shuffle (important for train/test split)
df = df.sample(frac=1.0, random_state=42).reset_index(drop=True)

In [9]:
# Map labels to ids
df["label_id"] = df["label"].map(label2id)

In [10]:
df.head(), df["label"].value_counts()

(                                              clause     label  \
 0  Communicate with you We use information we col...      none   
 1  To have any such information deleted, submit t...      none   
 2  Because many of our services won’t function wi...  tracking   
 3  For example, Uber uses identity verification t...      none   
 4  Car fleet information including capacities is ...      none   
 
                   source_policy  label_id  
 0               youtube_privacy         0  
 1                  hulu_privacy         0  
 2                paypal_privacy         2  
 3  uber_driversdelivery_privacy         0  
 4                 priceline_TOS         0  ,
 label
 none            11627
 data_sharing      768
 tracking          526
 refunds           134
 location           81
 arbitration        59
 ai_decisions       51
 Name: count, dtype: int64)

In [11]:
df.to_csv("../data/processed/fineprint_clauses_train_ready.csv", index=False)

In [12]:
df

,clause,label,source_policy,label_id
0,Communicate with you We use information we col...,none,youtube_privacy,0
1,"To have any such information deleted, submit t...",none,hulu_privacy,0
2,Because many of our services won’t function wi...,tracking,paypal_privacy,2
3,"For example, Uber uses identity verification t...",none,uber_driversdelivery_privacy,0
4,Car fleet information including capacities is ...,none,priceline_TOS,0
...,...,...,...,...
13241,Regional and State Privacy Disclosures For add...,none,ebay_privacy,0
13242,Disclosure of Personal Information We share yo...,none,taskrabbit_privacy,0
13243,This includes sending Snapchatters communicati...,none,snapchat_privacy,0
13244,"Rather, this Privacy Policy merely reflects gu...",none,united_privacy,0


# Train test split

In [15]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["label_id"],  # keeps label distribution similar
    random_state=42,
)

len(train_df), len(test_df), train_df["label"].value_counts(), test_df["label"].value_counts()

(10596,
 2650,
 label
 none            9301
 data_sharing     614
 tracking         421
 refunds          107
 location          65
 arbitration       47
 ai_decisions      41
 Name: count, dtype: int64,
 label
 none            2326
 data_sharing     154
 tracking         105
 refunds           27
 location          16
 arbitration       12
 ai_decisions      10
 Name: count, dtype: int64)